Long-term international migration 2.01a, citizenship, UK and England and Wales

In [1]:
from gssutils import *
scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'internationalmigration/datasets/longterminternationalmigrationcitizenshiptable201a')
scraper

## Long-term international migration 2.01a, citizenship, UK and England and Wales

Nationality of migrants. Estimates of Long-Term International Migration, annual table.

### Distributions

1. Long-term international migration 2.01a, citizenship, UK and England and Wales ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/longterminternationalmigrationcitizenshiptable201a/current/2.01altimcitizenship2004to2017.xls))


In [2]:
tab = next(t for t in scraper.distribution().as_databaker() if t.name == 'Table 2.01a')

Observations are in pairs of value +- confidence interval. Also, the table has been revised since the 2011 census and contains details about which observations have been revised and what their original estimates were.

In [3]:
observations = tab.filter('Year') \
    .shift(RIGHT) \
    .fill(DOWN) \
    .filter('Estimate') \
    .expand(RIGHT) \
    .filter('Estimate') \
    .fill(DOWN) \
    .is_not_blank() \
    .is_not_whitespace()
observations = observations - (tab.excel_ref('A1')
                               .expand(DOWN)
                               .filter(contains_string('Significant Change'))
                               .expand(RIGHT)
                              )
original_estimates = tab \
    .filter(contains_string('Original Estimates')) \
    .fill(DOWN) \
    .is_number()

observations = observations - original_estimates
savepreviewhtml([observations, original_estimates])

In [ ]:
CI = observations.shift(RIGHT)
Citizenship = tab.excel_ref('B12').expand(RIGHT).is_not_blank()
Citizenship_dim = HDim(Citizenship, 'Citizenship', CLOSEST, LEFT)
Citizenship_dim.AddCellValueOverride('European Union2', 'European Union')
Citizenship_dim.AddCellValueOverride('Non-European Union3', 'Non-European Union')
Region = tab.excel_ref('I13').expand(RIGHT).is_not_blank()
Region_dim = HDim(Region, 'Region', CLOSEST, LEFT)
Region_dim.AddCellValueOverride('European Union2', 'European Union')
Region_dim.AddCellValueOverride('All3', 'All')
Region_dim.AddCellValueOverride('Other Europe3', 'Other Europe')
Territory = tab.excel_ref('X14').expand(RIGHT).is_not_blank() - tab.excel_ref('AX14').expand(RIGHT).is_not_blank()
Geography = tab.excel_ref('A').expand(DOWN).one_of(['United Kingdom', 'England and Wales'])
Flow = tab.excel_ref('A').expand(DOWN).one_of(['Inflow', 'Outflow', 'Balance'])
Year = tab.filter('Year').fill(DOWN) & \
    observations.fill(LEFT) # tab.filter('Year').fill(DOWN).is_number() #excel_ref('A12').expand(DOWN) - Geography - Flow - tab.excel_ref('A116').expand(DOWN)
Revision = tab.excel_ref('AX14').expand(RIGHT).is_not_blank()

csObs = ConversionSegment(observations, [
    HDim(Year,'Year',DIRECTLY,LEFT),
    HDim(Geography,'Geography',CLOSEST,ABOVE),
    Citizenship_dim,
    Region_dim,
    HDim(Territory,'Territory',DIRECTLY,ABOVE),
    HDim(Flow,'Flow',CLOSEST,ABOVE),
    HDimConst('Measure Type', 'Count'),
    HDimConst('Unit','People (thousands)'),
    HDim(CI,'CI',DIRECTLY,RIGHT)
])
savepreviewhtml(csObs)

In [ ]:
tidy_revised = csObs.topandas()
tidy_revised

Also need to pull out the group of original estimates

In [ ]:
csRevs = ConversionSegment(original_estimates, [
    HDim(Year, 'Year', DIRECTLY, LEFT),
    HDim(Geography,'Geography',CLOSEST,ABOVE),
    HDimConst('Citizenship', 'All citizenships'),
    HDimConst('Region', None),
    HDimConst('Territory', None),
    HDim(Flow, 'Flow', CLOSEST, ABOVE),
    HDimConst('Measure Type', 'Count'),
    HDimConst('Unit','People (thousands)'),
    HDim(original_estimates.shift(RIGHT), 'CI', DIRECTLY, RIGHT),
    HDimConst('Revision', 'Original Estimate')
])
savepreviewhtml(csRevs)

In [ ]:
orig_estimates = csRevs.topandas()
orig_estimates

In [ ]:
tidy = pd.concat([tidy_revised, orig_estimates], axis=0, join='outer', ignore_index=True, sort=False)
tidy


In [ ]:
original_slice = tidy[tidy['Revision'] == 'Original Estimate']
tidy['Revision'] = tidy.apply(
    lambda row: '2011 Census Revision' if row['CI'] == ':' else 'Original Estimate',
    axis=1
)
tidy

Check each observation has a year and use ints.

In [ ]:
tidy['Year'] = tidy['Year'].apply(lambda x: pd.to_numeric(x, downcast='integer'))

Ignore data markers for now and ensure all observations are integers.
**Todo: figure out what to do with data markers.**

In [ ]:
import numpy as np
tidy['OBS'].replace('', np.nan, inplace=True)
tidy.dropna(subset=['OBS'], inplace=True)
tidy.drop(columns=['DATAMARKER'], inplace=True)
tidy.rename(columns={'OBS': 'Value'}, inplace=True)
tidy['Value'] = tidy['Value'].astype(int)
tidy

Join together Citizenship, Region and Territory to identify the place in the eventual hierarchy, calling it `LTIM Citizenship`

In [ ]:
tidy['LTIM Citizenship'] = tidy.apply(
    lambda row: ':'.join(
        [str(place).strip().replace('\n', ' ') for place in [row['Citizenship'], row['Region'], row['Territory']]
         if place != None and str(place).strip() != 'All']
    ), axis=1
).astype('category')
tidy.drop(columns=['Citizenship', 'Region', 'Territory'], inplace=True)
tidy['LTIM Citizenship'].cat.categories

Convert to codes

In [ ]:
tidy['LTIM Citizenship'].cat.categories = tidy['LTIM Citizenship'].cat.categories.map(pathify)
tidy['LTIM Citizenship'] = tidy['LTIM Citizenship'].cat.rename_categories({
    'non-european-union': 'non-european-union-all',
    'non-european-union-asia': 'non-european-union-asia-all'
})
tidy['LTIM Citizenship'].cat.categories

Replace geography with UK geo codes

In [ ]:
tidy['Flow'] = tidy['Flow'].astype('category')
tidy['Flow'].cat.categories = tidy['Flow'].cat.categories.map(pathify)

In [ ]:
tidy['Geography'] = tidy['Geography'].apply(lambda x: {
    'United Kingdom': 'K02000001',
    'England and Wales': 'K04000001'
}.get(x))

Todo: some values (estimations / CIs) have been rounded to zero and indicated with a `0~`, but this seems to be using conditional formatting of some kind and doesn't come through. We need to add data markers.

For CI we'll use a blank string for these markers, otherwise use the string representation of the int so it comes out in CSV okish.

In [ ]:
tidy['CI'] = tidy['CI'].apply(
    lambda x: '' if str(x) in ['', '0', ':', 'z'] else int(float(x))
)
tidy

Re-order the columns and output as CSV with some metadata.

In [ ]:
tidy = tidy[['Geography','Year','LTIM Citizenship','Flow','Measure Type','Value','CI','Unit', 'Revision']]
from pathlib import Path
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

tidy.to_csv(destinationFolder / ('observations.csv'), index = False)

scraper.dataset.family = 'migration'

with open(destinationFolder / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())